In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.cluster import KMeans

In [2]:
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Load images from dataset directory
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in glob.glob(os.path.join(folder, "*.jpg")):  # Adjust extension if needed
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
        if img is not None:
            img = cv2.resize(img, (64, 64))  # Resize to a fixed size
            images.append(img)
            labels.append(label)
    return images, labels

# Load dataset (Modify path according to your dataset)
cat_images, cat_labels = load_images_from_folder("dataset/cats", label=0)
dog_images, dog_labels = load_images_from_folder("dataset/dogs", label=1)

# Combine dataset
X = np.array(cat_images + dog_images)
y = np.array(cat_labels + dog_labels)

# Extract HOG features for each image
hog_features = []
for image in X:
    feature, _ = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    hog_features.append(feature)

X_hog = np.array(hog_features)

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_hog, y, test_size=0.2, random_state=42)

# Train an SVM classifier
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Predict on test data
y_pred = svm_model.predict(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


ModuleNotFoundError: No module named 'skimage'

In [3]:
import sys
!{sys.executable} -m pip install scikit-image


  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached imageio-2.37.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached tifffile-2025.3.13-py3-none-any.whl.metadata (32 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.8 MB 5.6 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/12.8 MB 6.5 MB/s eta 0:00:02
   ----------- ---------------------------- 3.7/12.8 MB 6.8 MB/s eta 0:00:02
   --------------- ------------------------ 5.0/12.8 MB 6.6 MB/s eta 0:00:02
   -------------------- ------------------- 6.6/12.8 MB 6.7 MB/s eta 0:00:01
   ------------------------ --------------- 7.9/12.8 MB 6.8 MB/s eta 0:00:01
   ----------------------------- ---------- 9.4/12.8 MB 6.7 MB/s eta 0:00:01
   --------------------------------- ------ 10.7/12.8 MB 6.6 MB/s eta 0:00:01
   ------------------------------------ --- 11.8/

  You can safely remove it manually.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Python38-32\python.exe -m pip install --upgrade pip


In [ ]:
rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), 30, 1)
test_image = cv2.warpAffine(test_image, rotation_matrix, (num_cols, num_rows))

test_gray = cv2.cvtColor(test_image, cv2.COLOR_RGB2GRAY)

# Display traning image and testing image
fx, plots = plt.subplots(1, 2, figsize=(20,10))

plots[0].set_title("Training Image")
plots[0].imshow(training_image)

plots[1].set_title("Testing Image")
plots[1].imshow(test_image)

# Detect keypoints and Create Descriptor

In [ ]:
# Initialize SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors for both images
keypoints_train, descriptors_train = sift.detectAndCompute(training_gray, None)
keypoints_test, descriptors_test = sift.detectAndCompute(test_gray, None)

# Use BFMatcher to match descriptors
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
matches = bf.knnMatch(descriptors_train, descriptors_test, k=2)

# Apply ratio test to filter matches
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

# If sufficient matches are found, proceed with homography
if len(good_matches) > 10:
    # Extract matching keypoints' coordinates
    src_pts = np.float32([keypoints_train[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints_test[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Compute homography matrix
    matrix, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    # Draw bounding box around detected object in the test image
    h, w = training_gray.shape
    corners = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
    transformed_corners = cv2.perspectiveTransform(corners, matrix)

    test_image_with_box = test_image.copy()
    test_image_with_box = cv2.polylines(test_image_with_box,
                                        [np.int32(transformed_corners)],
                                        isClosed=True,
                                        color=(255, 0, 0),
                                        thickness=3)

    # Display results
    plt.figure(figsize=(15, 15))
    plt.title("Detected Object")
    plt.imshow(test_image_with_box)
    plt.show()
else:
    print("Not enough matches found to detect the object.")

In [ ]:
# Initialize SIFT detector
sift = cv2.SIFT_create()
# Create Brute Force Matcher with L2 norm
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

# Perform KNN matching (find the two best matches for each descriptor)
matches = bf.knnMatch(train_descriptor, test_descriptor, k=2)

# Apply Lowe’s Ratio Test
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

# Sort matches based on distance
good_matches = sorted(good_matches, key=lambda x: x.distance)

# Find homography if enough matches are found
if len(good_matches) > 4:
    # Extract matching keypoints
    src_pts = np.float32([train_keypoints[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([test_keypoints[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Compute homography using RANSAC
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    # Get object bounding box in training image
    h, w = training_gray.shape
    obj_corners = np.float32([[0, 0], [w, 0], [w, h], [0, h]]).reshape(-1, 1, 2)

    # Project bounding box onto test image
    if H is not None:
        scene_corners = cv2.perspectiveTransform(obj_corners, H)
        
        # Draw bounding box
        test_image_color = cv2.cvtColor(test_gray, cv2.COLOR_GRAY2BGR)
        cv2.polylines(test_image_color, [np.int32(scene_corners)], True, (0, 255, 0), 3, cv2.LINE_AA)

        # Draw matches
        result = cv2.drawMatches(training_image, train_keypoints, test_image_color, test_keypoints,
                                 good_matches[:50], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        
        # Display result
        plt.figure(figsize=(14, 7))
        plt.title("Feature Matching & Object Detection")
        plt.imshow(result)
        plt.show()

        print("\nNumber of Filtered Matching Keypoints:", len(good_matches))
    else:
        print("Homography could not be computed.")
else:
    print("Not enough matches found.")

# Matching Keypoints

In [ ]:
# Create a BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

# Find matches using knn
matches = bf.knnMatch(train_descriptor, test_descriptor, k=2)

# Apply Lowe's ratio test
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

# Extract matched keypoints
train_pts = np.float32([train_keypoints[m.queryIdx].pt for m in good_matches])

# Apply KMeans to cluster keypoints into 4 groups (corresponding to 4 objects)
num_objects = 4
kmeans = KMeans(n_clusters=num_objects, random_state=42, n_init=10)
labels = kmeans.fit_predict(train_pts)

# Draw bounding boxes around clusters
train_image_copy = training_image.copy()
for i in range(num_objects):
    cluster_points = train_pts[labels == i]
    
    if len(cluster_points) < 5:  # Ignore very small clusters (noise)
        continue

    x_min, y_min = np.min(cluster_points, axis=0)
    x_max, y_max = np.max(cluster_points, axis=0)

    # Expand bounding box slightly
    padding = 20
    x_min, y_min = max(0, x_min - padding), max(0, y_min - padding)
    x_max, y_max = min(train_image_copy.shape[1], x_max + padding), min(train_image_copy.shape[0], y_max + padding)

    # Draw bounding box
    cv2.rectangle(train_image_copy, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 3)

# Display detected objects
plt.figure(figsize=(12, 6))
plt.imshow(cv2.cvtColor(train_image_copy, cv2.COLOR_BGR2RGB))
plt.title("Detected Objects in Training Image")
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def detect_and_match_objects(reference_image_path, query_image_path, min_match_count=10):
    """
    Detect objects in a query image based on a reference image using SIFT features.
    
    Parameters:
    reference_image_path (str): Path to the reference image (object to be detected)
    query_image_path (str): Path to the query image (where to find the object)
    min_match_count (int): Minimum number of good matches required
    
    Returns:
    tuple: (result_image, homography_matrix, matches_mask)
    """
    # Load images
    reference_img = cv2.imread(reference_image_path)
    query_img = cv2.imread(query_image_path)
    
    # Convert to grayscale
    reference_gray = cv2.cvtColor(reference_img, cv2.COLOR_BGR2GRAY)
    query_gray = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)
    
    # Initialize SIFT detector
    sift = cv2.SIFT_create()
    
    # Find keypoints and descriptors
    kp_reference, des_reference = sift.detectAndCompute(reference_gray, None)
    kp_query, des_query = sift.detectAndCompute(query_gray, None)
    
    # FLANN parameters for matching
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    
    # Create FLANN matcher
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    
    # Match descriptors
    matches = flann.knnMatch(des_reference, des_query, k=2)
    
    # Apply ratio test (Lowe's ratio test)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)
    
    # Create a mask for drawing matches
    matches_mask = [[0, 0] for _ in range(len(matches))]
    
    # Try to find homography if we have enough good matches
    homography = None
    if len(good_matches) >= min_match_count:
        # Extract location of good matches
        src_pts = np.float32([kp_reference[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_query[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        
        # Find homography
        homography, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        
        # Update matches_mask based on inliers
        for i, (m, _) in enumerate(matches):
            if m in good_matches and mask[good_matches.index(m)]:
                matches_mask[i] = [1, 0]
        
        # Draw bounding box around the detected object
        h, w = reference_gray.shape
        corners = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
        transformed_corners = cv2.perspectiveTransform(corners, homography)
        
        # Draw the bounding box
        result_img = query_img.copy()
        cv2.polylines(result_img, [np.int32(transformed_corners)], True, (0, 255, 0), 3)
        
        print(f"Object found - {len(good_matches)} good matches")
    else:
        print(f"Not enough matches found - {len(good_matches)}/{min_match_count}")
        result_img = query_img.copy()
        homography = None
    
    return result_img, homography, matches_mask

def visualize_keypoints(image_path):
    """
    Visualize SIFT keypoints on an image
    
    Parameters:
    image_path (str): Path to the image
    
    Returns:
    numpy.ndarray: Image with keypoints drawn
    """
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    sift = cv2.SIFT_create()
    keypoints = sift.detect(gray, None)
    
    img_with_keypoints = cv2.drawKeypoints(gray, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    return img_with_keypoints

def visualize_matches(reference_image_path, query_image_path):
    """
    Visualize matches between reference and query images
    
    Parameters:
    reference_image_path (str): Path to the reference image
    query_image_path (str): Path to the query image
    
    Returns:
    numpy.ndarray: Image showing matches
    """
    # Load images
    reference_img = cv2.imread(reference_image_path)
    query_img = cv2.imread(query_image_path)
    
    # Convert to grayscale
    reference_gray = cv2.cvtColor(reference_img, cv2.COLOR_BGR2GRAY)
    query_gray = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)
    
    # Initialize SIFT detector
    sift = cv2.SIFT_create()
    
    # Find keypoints and descriptors
    kp_reference, des_reference = sift.detectAndCompute(reference_gray, None)
    kp_query, des_query = sift.detectAndCompute(query_gray, None)
    
    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    
    # Create FLANN matcher
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    
    # Match descriptors
    matches = flann.knnMatch(des_reference, des_query, k=2)
    
    # Apply ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)
    
    # Draw matches
    img_matches = cv2.drawMatches(reference_img, kp_reference, query_img, kp_query, good_matches, None,
                                  flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    return img_matches

# Example usage
if __name__ == "__main__":
    # Replace these with your actual image paths
    reference_image = "ref.png"
    query_image = "shoe.jpg"
    
    # Detect and match objects
    result, homography, matches_mask = detect_and_match_objects(reference_image, query_image)
    
    # Convert result from BGR to RGB for displaying with matplotlib
    result_rgb = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
    
    # Visualize keypoints
    reference_keypoints = visualize_keypoints(reference_image)
    reference_keypoints_rgb = cv2.cvtColor(reference_keypoints, cv2.COLOR_BGR2RGB)
    
    # Visualize matches
    matches_visualization = visualize_matches(reference_image, query_image)
    matches_visualization_rgb = cv2.cvtColor(matches_visualization, cv2.COLOR_BGR2RGB)
    
    # Display results
    plt.figure(figsize=(15, 10))
    
    plt.subplot(2, 2, 1)
    plt.imshow(cv2.cvtColor(cv2.imread(reference_image), cv2.COLOR_BGR2RGB))
    plt.title('Reference Image')
    plt.axis('off')
    
    plt.subplot(2, 2, 2)
    plt.imshow(cv2.cvtColor(cv2.imread(query_image), cv2.COLOR_BGR2RGB))
    plt.title('Query Image')
    plt.axis('off')
    
    plt.subplot(2, 2, 3)
    plt.imshow(reference_keypoints_rgb)
    plt.title('SIFT Keypoints on Reference Image')
    plt.axis('off')
    
    plt.subplot(2, 2, 4)
    plt.imshow(result_rgb)
    plt.title('Object Detection Result')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Show matches in a separate figure
    plt.figure(figsize=(15, 10))
    plt.imshow(matches_visualization_rgb)
    plt.title('Feature Matches')
    plt.axis('off')
    plt.show()